# Latent Dirichlet Allocation

### NLP

Latent Dirichlet Allocation


* 00) Load Data
* 01) Preprocessing
* 02) LDA
* 03) WORD
* 04) Top Words per topic
* 05) Labels to document



## Data

This data has been picked from the NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [ ]:
# Libraries
import pandas as pd


00) Load Data

In [ ]:
"""
In this chunk we define the relative path for our dataset.
"""

from google.colab import drive

mount='/content/gdrive'
print("Colab: mounting Google drive on ", mount)

drive.mount(mount)

# Switch to the directory on the Google Drive that you want to use
import os
drive_root = mount + "/My Drive/NUEVO/Chap_6"

# Create drive_root if it doesn't exist
create_drive_root = True
if create_drive_root:
  print("\nColab: making sure ", drive_root, " exists.")
  os.makedirs(drive_root, exist_ok=True)

# Change to the directory
print("\nColab: Changing directory to ", drive_root)
%cd $drive_root

Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/NUEVO/Chap_6  exists.

Colab: Changing directory to  /content/gdrive/My Drive/NUEVO/Chap_6
/content/gdrive/My Drive/NUEVO/Chap_6


In [ ]:
npr = pd.read_csv('quora_questions.csv')
npr.head()

Unnamed: 0.1  Unnamed: 0                                           Question
0             0           0  What is the step by step guide to invest in sh...
1             1           1  What is the story of Kohinoor (Koh-i-Noor) Dia...
2             2           2  How can I increase the speed of my internet co...
3             3           3  Why am I mentally very lonely? How can I solve...
4             4           4  Which one dissolve in water quikly sugar, salt...

Como podemos ver no hay ninguna columna correspondiente al tema. Por esta razón decimos que es un modelo no supervisado. Nuestro objetivo principal es identificar los tópicos de cada documento usando LDA.

## 01) Preprocessing

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

The first step is to clean up the document as much as we can. We could apply stop words and TF-IDF approximation but instead of these steps we will use the CountVectorizer function.

The Count Vectorizer has two important arguments which are:

**`max_df`**` :We will ignore terms that have a document frequency strictly above a certain value. This value is what we call the threshold.

**`min_df`**` :This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

Hay muchos otros argumentos como stop_words que tendremos en cuenta para limpiar nuestros documentos.


In [ ]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(npr['Question'])
dtm

<50000x13573 sparse matrix of type '<class 'numpy.int64'>'
	with 238154 stored elements in Compressed Sparse Row format>

## 02) LDA
In this case we ask for 5 topics but we can search for as many topics as we want.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5,random_state=113)
# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)


LatentDirichletAllocation(n_components=5, random_state=113)

##03) Stored Words

In [ ]:
# Count Vectorizer vocabulary.
len(cv.get_feature_names_out())

13573

In [ ]:
# Show 10 random words from vocab
import random

for i in range(10):
    random_word_id = random.randint(0,7136) # This is a random probability
    print(cv.get_feature_names_out()[random_word_id])

cochin
escorts
30
closely
identify
labor
journalists
asia
chatting
arguments


##04) Showing Top Words Per Topic

In [ ]:
len(LDA.components_)

5

In [ ]:
LDA.components_ # As we can se we have 5 components per document. In case we had 7, we would have 7 components per document.

array([[ 0.20084156, 26.90441857,  0.20387162, ...,  1.19966133,
         3.19426531,  2.19745559],
       [ 3.57630775,  0.20478496,  1.20130051, ...,  0.20000821,
         0.20062671,  0.20001456],
       [ 0.20035024, 30.44926222,  0.20001628, ...,  0.20000837,
         0.20282897,  0.20001483],
       [ 0.20197175, 28.54428178,  0.2170473 , ...,  1.20031309,
         0.2022654 ,  0.20249872],
       [ 2.8205287 ,  7.89725246,  1.17776429, ...,  0.200009  ,
         0.20001362,  0.2000163 ]])

These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [ ]:
for index,topic in enumerate(LDA.components_): # for each topic grab the most representive words
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['book', 'free', 'programming', 'job', 'india', 'like', 'lose', 'books', 'language', 'weight', 'engineering', 'good', 'learn', 'way', 'best']


THE TOP 15 WORDS FOR TOPIC #1
['person', 'math', 'question', 'sex', 'black', 'feel', 'like', 'questions', 'love', 'mean', 'make', 'money', 'quora', 'people', 'does']


THE TOP 15 WORDS FOR TOPIC #2
['don', 'facebook', 'hillary', 'iphone', 'did', 'movie', 'long', 'india', 'things', 'account', 'like', 'new', 'day', 'know', 'does']


THE TOP 15 WORDS FOR TOPIC #3
['new', 'world', 'war', 'old', 'donald', 'notes', '1000', '500', 'think', 'year', 'start', 'did', 'indian', 'trump', 'india']


THE TOP 15 WORDS FOR TOPIC #4
['english', 'examples', 'work', 'meaning', 'difference', 'increase', 'energy', 'word', 'thing', 'used', 'improve', 'good', 'time', 'life', 'does']




We have 5 topics.
  * Topic 0 politics
  * Topic 1 Music
  * Topic 2 Goverment
  * Topic 3 Education
  * Topic 4 Military


## 5) Attaching Discovered Topic Labels to Original Articles

In [ ]:
dtm

<50000x13573 sparse matrix of type '<class 'numpy.int64'>'
	with 238154 stored elements in Compressed Sparse Row format>

In [ ]:
# Case for the first document

topic_results= LDA.transform(dtm)
topic_results[0]

array([0.02561432, 0.02565666, 0.02539872, 0.89724273, 0.02608758])

We can see that document "0" has 5 arguments. The number or argument correspond to the number of document that we have asked for. In this case we would have to format the number of decimals and after that que would keep just the maximun value. This would be the represent the topic.

In [ ]:
topic_results[0].round(2)

array([0.03, 0.03, 0.03, 0.9 , 0.03])

In [ ]:
topic_results[0].argmax()

3

In [ ]:
npr.iloc[0,0]

0

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [ ]:
npr.head()

Unnamed: 0.1  Unnamed: 0                                           Question
0             0           0  What is the step by step guide to invest in sh...
1             1           1  What is the story of Kohinoor (Koh-i-Noor) Dia...
2             2           2  How can I increase the speed of my internet co...
3             3           3  Why am I mentally very lonely? How can I solve...
4             4           4  Which one dissolve in water quikly sugar, salt...

In [ ]:
topic_results.argmax(axis=1)

array([3, 3, 0, ..., 1, 2, 2])

In [ ]:
npr['Topic'] = topic_results.argmax(axis=1)

In [ ]:
npr.head(10)

Unnamed: 0.1  Unnamed: 0  \
0             0           0   
1             1           1   
2             2           2   
3             3           3   
4             4           4   
5             5           5   
6             6           6   
7             7           7   
8             8           8   
9             9           9   

                                            Question  Topic  
0  What is the step by step guide to invest in sh...      3  
1  What is the story of Kohinoor (Koh-i-Noor) Dia...      3  
2  How can I increase the speed of my internet co...      0  
3  Why am I mentally very lonely? How can I solve...      0  
4  Which one dissolve in water quikly sugar, salt...      4  
5  Astrology: I am a Capricorn Sun Cap moon and c...      1  
6                                Should I buy tiago?      0  
7                     How can I be a good geologist?      0  
8                    When do you use シ instead of し?      1  
9  Motorola (company): Can I hack my Charter Moto...      2